In [40]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [41]:
class AITextDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

In [53]:
data = pd.read_pickle("feature_engineered_embeddings")
shuffled = data.sample(frac = 1)
splits = np.array_split(shuffled, 2)

/Users/devammondal/PycharmProjects/lstm/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [67]:
train = splits[0]
test = splits[1]

In [68]:
print(train)

                                                    text  generated  \
26673  Everyday, millions of people around the world ...          1   
23557  There is no definitive answer to this question...          1   
22025  There is no denying that medical care and educ...          1   
26917  We all want to make a positive impact on the w...          1   
16527  DISTANCE LEARNING\n\nI think students can be b...          0   
...                                                  ...        ...   
18705  \nFamously competitive rivals, Cristiano Ronal...          1   
23126  There is no denying that facts are important. ...          1   
19088  \nCaring for someone with mental health issues...          1   
20046  \nHaving a positive attitude in a difficult si...          1   
8272   How to know how a student feels in your class?...          0   

                                              embeddings  
26673  [[tensor(0.1263), tensor(-0.1514), tensor(-0.3...  
23557  [[tensor(0.1713), tens

In [69]:
train_dataset = AITextDataset(data=list(train.embeddings), labels=list(train.generated))
test_dataset = AITextDataset(data=list(test.embeddings), labels=list(train.generated))

In [70]:
class LSTM(torch.nn.Module):
    def __init__(self, input, hidden, layers, output):
        super(LSTM, self).__init__()
        self.hidden = hidden
        self.layers = layers
        self.lstm = torch.nn.LSTM(input, hidden, layers, output, batch_first = True)
        self.connected = torch.nn.Linear(hidden, output)
        
    def forward(self, x):
        hidden_state = torch.zeros(self.layers, x.size(0), self.hidden)
        memory = torch.zeros(self.layers, x.size(0), self.hidden)
        out, _  = self.lstm(x, (hidden_state, memory))
        out = self.fc(out[:, -1, :])
        return out

In [71]:
model = LSTM(768, 64, 2, 2)

In [72]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters())

In [73]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle = True)

In [76]:
EPOCHS = 100
train_loss = []
train_acc = []
test_acc = []

for epoch in range(EPOCHS):
    for i, data in enumerate(train_loader):
        tensor, label = data
        
        prediction = model(tensor)
        loss = loss_function(prediction, label)
        accuracy = (prediction.round() == label).float().mean()
        
        train_loss.append(loss)
        train_acc.append(accuracy)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        print(f"epoch {epoch} step {i} loss {loss} accuracy {accuracy}")

TypeError: lstm() received an invalid combination of arguments - got (Tensor, tuple, list, int, int, float, bool, bool, bool), but expected one of:
 * (Tensor data, Tensor batch_sizes, tuple of Tensors hx, tuple of Tensors params, bool has_biases, int num_layers, float dropout, bool train, bool bidirectional)
      didn't match because some of the arguments have invalid types: (Tensor, !tuple of (Tensor, Tensor)!, !list of [Parameter, Parameter, Parameter, Parameter, Parameter, Parameter, Parameter, Parameter]!, !int!, !int!, !float!, !bool!, bool, bool)
 * (Tensor input, tuple of Tensors hx, tuple of Tensors params, bool has_biases, int num_layers, float dropout, bool train, bool bidirectional, bool batch_first)
      didn't match because some of the arguments have invalid types: (Tensor, !tuple of (Tensor, Tensor)!, !list of [Parameter, Parameter, Parameter, Parameter, Parameter, Parameter, Parameter, Parameter]!, !int!, int, float, bool, bool, bool)
